In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Reshape, LeakyReLU, BatchNormalization, Conv2D, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.datasets import mnist

In [2]:
# Load and preprocess data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train / 127.5 - 1.0  # Normalize images to [-1, 1]
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension

# Normalize and reshape the data
X_train = (X_train.astype(np.float32) - 127.5) / 127.5  # Rescale to [-1, 1]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)  # Reshape to (28, 28, 1)

X_test = (X_test.astype(np.float32) - 127.5) / 127.5  # Rescale to [-1, 1]
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)  # Reshape to (28, 28, 1)

In [3]:
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(Dense(28 * 28, activation='tanh'))
    model.add(Reshape((28, 28, 1)))  # Reshape to 28x28 image
    return model

In [4]:
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=(28, 28, 1)))
    model.add(LeakyReLU(0.2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

In [5]:
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator when training GAN
    gan_input = Input(shape=(latent_dim,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(gan_input, gan_output)
    return gan

In [6]:
def train_gan(generator, discriminator, gan, images, latent_dim, batch_size, epochs):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, images.shape[0], half_batch)
        real_images = images[idx]
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_images = generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(real_images, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
        
        print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")
        
        # Display images every 1000 epochs
        if epoch % 1000 == 0:  # Show every 1000 epochs
            noise = np.random.normal(0, 1, (1, latent_dim))
            generated_image = generator.predict(noise)
            plt.imshow(generated_image[0, :, :, 0], cmap='gray')
            plt.show()

In [9]:
image_dir = 'C:/Users/Ahmadee/PycharmProjects/generative-design-project/data/raw_images'
images = load_images(image_dir)
X_train = (images - 127.5) / 127.5
X_train = X_train.astype (np.float32)

latent_dim = 100  # Latent dimension for random noise
batch_size = 64
epochs = 10000

# Initialize the models
generator = build_generator(latent_dim)
discriminator = build_discriminator()

# Compile the models
discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])
gan = build_gan(generator, discriminator)
gan.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

# Train the GAN
train_gan(generator, discriminator, gan, X_train, latent_dim, batch_size, epochs)

NameError: name 'cv2' is not defined

In [10]:
def load_images(image_dir, target_size=(28, 28)):
    images = []
    # Loop through all files in the directory
    for filename in os.listdir(image_dir):
        img_path = os.path.join(image_dir, filename)
        
        # Load image using OpenCV or Keras' image module
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # Resize image to the desired target size (e.g., 28x28 for MNIST-like dataset)
        img = cv2.resize(img, target_size)
        
        # Normalize the image to [-1, 1] range and reshape
        img = (img.astype(np.float32) - 127.5) / 127.5
        img = np.expand_dims(img, axis=-1)  # Add channel dimension
        
        images.append(img)
    
    # Convert list to numpy array
    images = np.array(images)
    return images

# 2. Now, load the images (make sure to call this after the function is defined)
image_dir = 'C:/Users/Ahmadee/PycharmProjects/generative-design-project/data/raw_images'
images = load_images(image_dir)

# 3. Preprocess the images
X_train = images  # This holds your loaded and processed images

NameError: name 'cv2' is not defined